### **Data Mining Using SQL**

In [ ]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [ ]:
# Choose files Shop.csv and Movies.csv to upload - must be on local computer
# If running notebook on local computer:
#   No need to run this cell (it will generate an error)
#   Make sure data files are in same workspace as notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load tables from CSV files
# Shop
with open('Shop.csv') as f: Shop = pd.read_csv(f, index_col=0)
%sql drop table if exists Shop;
%sql persist Shop
# Movies
with open('Movies.csv') as f: Movies = pd.read_csv(f, index_col=0)
%sql drop table if exists Movies;
%sql persist Movies

## Shopping dataset - frequent item-sets

In [ ]:
%%sql
select * from Shop

### Frequent item-sets of two

#### All pairs of items occurring together

In [ ]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item

#### Now with number of transactions they occur together in

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item

#### Total number of transactions

In [ ]:
%%sql
select count(distinct TID) from Shop

#### Put it together for frequent item-sets

In [ ]:
support = .3
print(support)

In [ ]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

### Frequent item-sets of three

#### All triples of items occurring together

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item

#### Now with number of transactions they occur together in

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3, count()
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item

#### Put it together for frequent item-sets

In [ ]:
support = .3
print(support)

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

### <font color = 'green'>**Your Turn - Movies dataset frequent item-sets**</font>

In [ ]:
%%sql
select * from Movies limit 15

In [ ]:
%%sql
select count(distinct TID) as numUsers,
       count(distinct item) as numMovies
from Movies

#### Mine for frequent item-sets of three and four items in the Movies dataset. Find a single support threshold where the number of frequent item-sets of three items is more than 10 but less than 20, and the number of frequent item-sets of four items is more than 0.

In [ ]:
support = INSERT VALUE HERE
print(support)

In [ ]:
%%sql
INSERT CODE FOR ITEM-SETS OF THREE HERE

In [ ]:
%%sql
INSERT CODE FOR ITEM-SETS OF FOUR HERE

## Shopping dataset - association rules

### Association rules with one item on the left-hand side

In [ ]:
support = .5
confidence = .5
print(support, confidence)

#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules; include number of transactions

In [ ]:
%%sql
select item, count() as numtrans
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

#### Do the same query but save the result in a table called "Frequents"

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

#### Add check for specified confidence to get final result

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > :confidence

### Association rules with two items on the left-hand side

In [ ]:
support = .5
confidence = .5
print(support, confidence)

#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

#### Add check for specified confidence to get final result

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / F.numtrans > :confidence

## Shopping dataset - association rules with lift instead of confidence

#### Compute overall frequency of each item -- needed for lift calculation

In [ ]:
%%sql
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item

#### Do the same query but save the result in a table called "ItemFreq"

In [ ]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item;
select * from ItemFreq

### Association rules with one item on the left-hand side

In [ ]:
support = .5
print(support)

#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions.

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

#### Add check for lift > 1 to get final result

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Frequent F, ItemFreq I
where T1.item = F.item
and T1.TID = T2.TID
and T2.item = I.item
and T1.item != T2.item
group by T1.item, T2.item
having (1.0*count() / F.numtrans) / I.freq > 1

### Association rules with two items on the left-hand side

In [ ]:
support = .5
print(support)

#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

#### Add check for lift > 1 to get final result

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Shop T3, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID and T3.item = I.item
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having (1.0*count() / F.numtrans) / I.freq > 1

### <font color = 'green'>**Your Turn - Movies dataset association rules**</font>

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and confidence thresholds (need not be the same) so the number of association rules is more than 10 but less than 20.


In [ ]:
support = INSERT VALUE HERE
confidence = INSERT SAME OR DIFFERENT VALUE HERE
print(support, confidence)

In [ ]:
%%sql
drop table if exists Frequent;
INSERT CODE FOR CREATING FREQUENT TABLE WITH ITEM-SETS OF THREE HERE

In [ ]:
%%sql
INSERT CODE FOR ASSOCIATION RULES HERE

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and lift thresholds so the number of association rules is more than 10 but less than 20.


In [ ]:
support = INSERT VALUE HERE
lift = INSERT VALUE HERE
print(support, lift)

In [ ]:
%%sql
drop table if exists ItemFreq;
INSERT CODE FOR CREATING TABLE WITH OVERALL FREQUENCY OF EACH ITEM

In [ ]:
%%sql
drop table if exists Frequent;
INSERT CODE FOR CREATING FREQUENT TABLE WITH ITEM-SETS OF THREE HERE

In [ ]:
%%sql
INSERT CODE FOR ASSOCIATION RULES HERE